In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Read company info into dataframe

csv_file = "Resources/securities.csv"
company = pd.read_csv(csv_file)
company.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [4]:
# Filter companies in health care sector
company_hc = company.loc[company["GICS Sector"] == "Health Care"].reset_index(drop=True)

final_hc_company = company_hc[["Ticker symbol","Security","GICS Sector"]].copy()

final_hc_company.head()

,Ticker symbol,Security,GICS Sector
0,ABT,Abbott Laboratories,Health Care
1,ABBV,AbbVie,Health Care
2,AET,Aetna Inc,Health Care
3,A,Agilent Technologies Inc,Health Care
4,ALXN,Alexion Pharmaceuticals,Health Care


In [109]:
company_ticker = final_hc_company[["Ticker symbol"]].copy()
company_ticker.head()

,Ticker symbol
0,ABT
1,ABBV
2,AET
3,A
4,ALXN


In [6]:
# Read company financials into dataframe

csv_file = "Resources/fundamentals.csv"
company_financial = pd.read_csv(csv_file)
company_financial.head()

#TO DO - 
#Inner join with company_ticker by ticker
#Filter certain columns based on google doc

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [7]:
# Read company historical prices into dataframe
csv_file = "Resources/prices.csv"
company_prices = pd.read_csv(csv_file)
company_prices.head()

#TO DO - 
#Inner join with company_ticker by ticker
#Filter 2016 years and calculate company average

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [103]:
from config import api_key
import requests
import json
import time

In [104]:
ticker_list = company_ticker["Ticker symbol"].to_list()

ticker_symbol_list = [ ]
date_list = [ ]
close = [ ]

for ticker in ticker_list:
    base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='
    url = f'{base_url}{ticker}&apikey={api_key}'
    response = requests.get(url).json()
    
    try:
        ticker_symbol = response['Meta Data']['2. Symbol']
        response_dict = response['Monthly Adjusted Time Series']
    
    except KeyError:
        print(response)
        continue
    
    for date in response_dict.keys():
        if '2019' in date:
            ticker_symbol_list.append(ticker_symbol)
            date_list.append(date)
            close.append(response_dict[date]['4. close'])
    time.sleep(12)

['ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABT', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'ABBV', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'ALXN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'AGN', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'AMGN', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'ANTM', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BAX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BDX', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BIIB', 'BSX', 'BSX', 'BSX', 'BSX', 'BSX', 'BSX

NameError: name 'close_list' is not defined

In [120]:
df_2019 = pd.DataFrame({ "Ticker symbol":ticker_symbol_list, 
                        "Date":date_list, 
                        "Close Price":close})
df_2019["Close Price"] = pd.to_numeric(df_2019["Close Price"])
df_2019

,Ticker symbol,Date,Close Price
0,ABT,2019-12-31,86.86
1,ABT,2019-11-29,85.45
2,ABT,2019-10-31,83.61
3,ABT,2019-09-30,83.67
4,ABT,2019-08-30,85.32
...,...,...,...
654,ZTS,2019-05-31,101.05
655,ZTS,2019-04-30,101.84
656,ZTS,2019-03-29,100.67
657,ZTS,2019-02-28,94.23


In [121]:
#TO DO - loop through tickers to get historical 2019 monthly stock prices(close) and 
#calculate average by company - avg_prices_2019
avg_prices_2019 = df_2019.groupby(['Ticker symbol']).mean()
avg_prices_2019

,Close Price
Ticker symbol,
A,76.255833
ABBV,77.736667
ABC,82.844167
ABT,81.861667
AGN,158.764167
ALXN,117.811667
AMGN,197.935000
ANTM,280.827500
BAX,80.154167


In [ ]:
#tables to load to database: each contains ticker columns which be used to join the tables within data base
#final_hc_company - done
#avg_prices_2019 - steph
#company_financials - serene
#avg_prices_2016 - young

#serene
    #set up database in postgres
    #connect to database and load the tables
    #join tables within postgres